# Élivágar: Efficient Quantum Circuit Search for Classification

In this notebook, we will explore using Élivágar, an efficient method for performing Quantum Circuit Search (QCS) for classification tasks on quantum computers. We will apply Élivágar to find performant, noise-robust circuits that can be used to classify MNIST digits on the OQC Lucy quantum computer, which is accessible via Amazon Braket.

## About Élivágar
Élivágar is a QCS framework that differs from prior QCS works such as QuantumNAS by using training- and gradient-free strategies to select suitable circuits for classification tasks. Since gradient computation is expensive on quantum computers due to the high cost of methods such as the parameter shift rule, QuantumNAS and other methods which use SuperCircuit-inspired setups requiring training incur high runtime costs. In contrast, Élivágar is much faster due to using gradient-free performance predictors to evaluate candidate circuits. 

Furthermore, Élivágar also incorporates device-aware circuit generation, which eliminates the need to perform a costly circuit-mapping co-search to find good qubit mappings for candidate circuits onto the target device. Élivágar instead generates high-quality qubit mappings before generating candidate circuits, and chooses operations that obey the connectivity constraints of the select mappings. Finally, Élivágar also searches over data embeddings, which is not supported in other QCS frameworks. This leads to significant performance gains on various classification tasks, since, as shown by multiple works in the Quantum Machine Learning (QML) literature, the embedding used with a circuit plays an extremely important role in determining circuit performance.

## Setting up Amazon Braket on your local development environment

Before you can follow along with the steps performed in this notebook, you will need to have your local development environment set up in Amazon Braket. If you have not done so already, you can follow the guide at [https://aws.amazon.com/blogs/quantum-computing/setting-up-your-local-development-environment-in-amazon-braket/](https://aws.amazon.com/blogs/quantum-computing/setting-up-your-local-development-environment-in-amazon-braket/) to do so.

## Required imports

First, we will import all of the packages and modules that we require for the demonstration:

In [50]:
import os
import numpy as np

from braket.aws import AwsDevice

from elivagar.circuits.device_aware import (
    extract_properties_from_braket_device,
    generate_device_aware_gate_circ
)

from elivagar.metric_computation.compute_clifford_nr import (
    compute_clifford_nr_for_circuits
)

from elivagar.metric_computation.compute_rep_cap import (
    compute_rep_cap_for_circuits
)

from elivagar.training.train_circuits_from_predictors import (
    train_elivagar_circuits
)

In [51]:
from importlib import reload

import elivagar.circuits.create_circuit
import elivagar.metric_computation.compute_clifford_nr
import elivagar.metric_computation.compute_rep_cap
import elivagar.circuits.device_aware

reload(elivagar.circuits.create_circuit)
reload(elivagar.metric_computation.compute_clifford_nr)
reload(elivagar.metric_computation.compute_rep_cap)
reload(elivagar.circuits.device_aware)

<module 'elivagar.circuits.device_aware' from 'D:\\Quantum_Computing\\Elivagar\\Elivagar\\elivagar\\circuits\\device_aware.py'>

## Get target device information

Before we can begin the process of generating candidate circuits for the MNIST task, we must first obtain the properties of the target device, in this case OQC Lucy. We require this because Elivagar generates candidate circuits in a device-aware way, using the differences in edge and readout fidelities as well as qubit coherence times to determine the best circuit structures to generate, and the best logical-to-physical qubit mappings for each generated candidate circuit. 

We will start by getting the device details from OQC Lucy using the Braket SDK:

In [14]:
# oqc_lucy_dev = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")
ionq_harmony_dev = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Harmony")

## Generate candidate circuits

In order to generate candidate circuits for a dataset, we first need to obtain some basic information about the dataset. IN our case, the dataset is MNIST-2, a subset of the MNIST handwritten digit dataset consisting of 2 classes, 0 and 1. While we can apply Élivágar to larger datasets, such as the whole MNIST dataset, as well, for the purposes of this tutorial, we will only use a smaller subset to keep things short.

We will search for a circuit with 4 qubits and 32 parameters. Since we will be using a preprocessed version of the MNIST dataset, the dimensionality of each sample is only 16, rather than the 784 in the original dataset. Additionally, since there are only two classes, we only need to measure 1 qubit - we can simply train the model to perform binary classification, i.e. output expectation values of either -1 or 1, depending on which class the input belongs to.

In [15]:
num_qubits = 4
num_meas_qubits = 1
num_params = 32
num_embeds = 16

We also need to specify some options related to the circuit generation process:
* num_circs: the number of candidate circuits to generate.
* add_rotations: whether to add RX, RY, and RZ single qubit gates to the gateset of the circuits beign generated, or use only device-native gates. Adding rotation gates often improves performance due to greater circuit expressivity, and does not affect the qubit mappings since they are all single qubit gates.
* param_focus: a number that specifies how important it is to generate gates that contain variational parameters versus non-paametrized gates. Increasing this will make the generated circuits contain fewer gates and result in reduced circuit depth, although it may reduce circuit expressivity.
* num_trial_mappings: the number of mappings to generate before beginning the candidate generation process. Increasing this number will lead to the selection of more high-quality mappings, but will also increase circuit generation time.
* temp: the temperate for various softmax distributions used by the circuit generation process. Increasing this results in higher circuit diversity, as different gates, gate placements, and qubit mappings are chosen with more uniform probabilities.

For now, we will use sensible defaults for each of these values, although you can try out different values for each to see how they affect downstream performance.

In [16]:
num_circs = 250
add_rotations = True
param_focus = 2.0
num_trial_mappings = 200
temp = 0.25

Now, we can generate the candidate circuits. For each circuit, we randomly sample a ent_prob value, which determines what the proportion of entangling (2-qubit) gates in the circuit will be. Higher values can lead to better performance, but also lower circuit fidelity. We will discuss this effect of this in more detail in the next section, which focuses on estimating the noise robustness of generated candidate circuits.

In [20]:
circuits_save_folder = './candidate_circuits'

if not os.path.exists(circuits_save_folder):
    os.mkdir(circuits_save_folder)

for i in range(num_circs):
    curr_circ_folder = os.path.join(circuits_save_folder, f'circ_{i + 1}')
        
    if not os.path.exists(curr_circ_folder):
        os.makedirs(curr_circ_folder)
            
    ent_prob = 0.3 + 0.5 * np.random.sample()
        
    (
        circ_gates, gate_placements, inputs_bounds, weights_bounds,
        selected_mapping, meas_qubits
    ) = generate_device_aware_gate_circ(
        None, num_qubits, num_embeds, num_params,
        ent_prob, add_rotations, param_focus,
        num_meas_qubits, num_trial_mappings,
        temp, braket_device_properties=ionq_harmony_dev.properties,
        braket_device_name='ionq_harmony'
    )

    np.savetxt(
        os.path.join(curr_circ_folder, 'gate_params.txt'),
        np.array(gate_placements, dtype=object), fmt="%s"
    )

    np.savetxt(os.path.join(curr_circ_folder, 'gates.txt'), circ_gates, fmt="%s")
    np.savetxt(os.path.join(curr_circ_folder, 'inputs_bounds.txt'), inputs_bounds)
    np.savetxt(os.path.join(curr_circ_folder, 'weights_bounds.txt'), weights_bounds)
    np.savetxt(os.path.join(curr_circ_folder, 'qubit_mapping.txt'), selected_mapping)
    np.savetxt(os.path.join(curr_circ_folder, 'meas_qubits.txt'), meas_qubits)

Notice that when a circuit is generated, we not only receive information about the circuit, such as the gates and gate placements, but also the qubit mapping, and even which qubits we should measure at the end of the circuit.

## Compute Clifford noise resilience for candidate circuits
After we finish generating candidate circuits, we want to rank them in some way that will enable us to choose the best circuit for the classification task we have in mind. The first step Élivágar uses to do this is to estimate how noise-robust each circuit is, which quantifies the degree to which circuit outputs remain unaffected by hardware errors occuring during circuit execution. This is a crucial component of predicting circuit performance on noisy hardware, since while a circuit might perform well on a noiseless somulator, it might be execssively deep, or contain many low-fidelity two-qubit gates, reducing performance when deployed on error-prone quantum hardware.

Note: this step involves running circuits on the OQC Lucy device on the cloud, and the total time it takes will be subject to queueing delays, making it difficult to predict an expected runtime. Computing CNR with 32 Clifford replicas for 250 circuits involves running 8000 circuits on the device, which is likely to take at least a few hours. You can reduce this time by changing the number of candidate circuits to consider to a smaller value.

In [21]:
circuits_save_folder = './candidate_circuits'
num_circs = 250
device_name = 'ionq_harmony'
num_qubits = 4
num_clifford_replicas = 32
num_shots = 2048
compute_actual_fidelity = False
num_trial_parameters = 128
dataset = None
encoding_type = 'angle'
num_data_reps = 1
use_qubit_mapping = True
save_cnr_scores = True
circ_index_offset = 0
dataset_file_extension = 'txt'
use_real_backend = False
use_qiskit = True

In [22]:
compute_clifford_nr_for_circuits(
    circuits_save_dire_folder, num_circs, device_name, num_qubits,
    num_clifford_replicas, num_shots, compute_actual_fidelity, num_trial_parameters,
    dataset, encoding_type, num_data_reps,
    None, use_qiskit, None, None, use_qubit_mapping, save_cnr_scores,  
    None, circ_index_offset, dataset_file_extension,
    use_real_backend
)

0 0.9726715087890625
1 0.9785003662109375
2 0.97552490234375
3 0.97454833984375
4 0.9777984619140625
5 0.967498779296875
6 0.977020263671875
7 0.9721221923828125
8 0.9700469970703125
9 0.976531982421875
10 0.9752655029296875
11 0.9711456298828125
12 0.9761199951171875
13 0.9782867431640625
14 0.98236083984375
15 0.9711151123046875
16 0.970458984375
17 0.96502685546875
18 0.9728851318359375
19 0.9725189208984375
20 0.9759368896484375
21 0.9751739501953125
22 0.9730377197265625
23 0.9744415283203125
24 0.9744873046875
25 0.97259521484375
26 0.97119140625
27 0.9764404296875
28 0.9725189208984375
29 0.96954345703125
30 0.9738311767578125
31 0.9753265380859375
32 0.9742431640625
33 0.973419189453125
34 0.9751129150390625
35 0.9803314208984375
36 0.9700164794921875
37 0.968292236328125
38 0.9719696044921875
39 0.9793701171875
40 0.97088623046875
41 0.975738525390625
42 0.9731903076171875
43 0.9727020263671875
44 0.973541259765625
45 0.9696197509765625
46 0.98016357421875
47 0.981094360351562

[0.9726715087890625,
 0.9785003662109375,
 0.97552490234375,
 0.97454833984375,
 0.9777984619140625,
 0.967498779296875,
 0.977020263671875,
 0.9721221923828125,
 0.9700469970703125,
 0.976531982421875,
 0.9752655029296875,
 0.9711456298828125,
 0.9761199951171875,
 0.9782867431640625,
 0.98236083984375,
 0.9711151123046875,
 0.970458984375,
 0.96502685546875,
 0.9728851318359375,
 0.9725189208984375,
 0.9759368896484375,
 0.9751739501953125,
 0.9730377197265625,
 0.9744415283203125,
 0.9744873046875,
 0.97259521484375,
 0.97119140625,
 0.9764404296875,
 0.9725189208984375,
 0.96954345703125,
 0.9738311767578125,
 0.9753265380859375,
 0.9742431640625,
 0.973419189453125,
 0.9751129150390625,
 0.9803314208984375,
 0.9700164794921875,
 0.968292236328125,
 0.9719696044921875,
 0.9793701171875,
 0.97088623046875,
 0.975738525390625,
 0.9731903076171875,
 0.9727020263671875,
 0.973541259765625,
 0.9696197509765625,
 0.98016357421875,
 0.9810943603515625,
 0.970733642578125,
 0.9750671386718

## Compute representational capacity for candidate circuits
Once we have computed CNR for each of the candidate circuits, we can move to predicting the performance of each circuit on the target dataset. Élivágar does this using Representational Capacity (RepCap), a gradient-free metric that enables accurate estimation of circuit performance. To compute RepCap for a circuit, we need to define a few hyperparameters, listed below:
* num_param_samples: the number of randomly sampled parameter sets to use with the circuit while computing RepCap. More samples will lead to more accurate predictions, but experiments have shown that even as few as 32 parameter sets can be enough.
* num_samples_per_class: the number of samples to choose from each class in the target dataset for using to compute RepCap. Choosing more samples leads to more accurate predictions, but more than 16 samples per class are not required for simple datasets such as MNIST-2.

In [46]:
circuits_save_dir = './candidate_circuits'
num_circs = 250
circ_prefix = 'circ'
num_qubits = 4
num_meas_qubits = 1
dataset = 'mnist_2'
num_classes = 2
num_samples_per_class = 16
num_param_samples = 32
encoding_type = 'angle'
num_data_reps = 1
save_matrices = False
dataset_file_extension = 'txt'

In [47]:
compute_rep_cap_for_circuits(
    circuits_save_dir, num_circs, circ_prefix, num_qubits, 
    num_meas_qubits, dataset, num_classes,
    num_samples_per_class,
    num_param_samples, encoding_type, 
    num_data_reps, save_matrices,
    dataset_file_extension
)

[[-1.]
 [ 1.]] (1600, 1)
0.8209934234619141
0.8502101898193359
0.6621513366699219
0.8642520904541016
0.9496726989746094
0.7711677551269531
0.8501815795898438
0.9283313751220703
0.9201908111572266
0.7827053070068359
0.8208694458007812
0.7746543884277344
0.846588134765625
0.8055324554443359
0.8881587982177734
0.8429241180419922
0.9579677581787109
0.8012771606445312
0.8541412353515625
0.7585334777832031
0.8044300079345703
0.8340282440185547
0.8056850433349609
0.8393459320068359
0.7680873870849609
0.8564529418945312
0.8788700103759766
0.7046527862548828
0.7756462097167969
0.6630420684814453
0.8422775268554688
0.7719173431396484
0.6940860748291016
0.9090404510498047
0.7060508728027344
0.757720947265625
0.8512344360351562
0.8611316680908203
0.7894706726074219
0.6631050109863281
0.8568859100341797
0.8999347686767578
0.5892105102539062
0.8155269622802734
0.8405666351318359
0.8009395599365234
0.8895053863525391
0.9194889068603516
0.7470378875732422
0.8480644226074219
0.9103069305419922
0.772651

## Compute composite scores for candidate circuits and train best circuit

Now that we have computed both CNR and RepCap scores for the candidate circuits, we can combine the two to create composite scores for each circuit. To decide the relative importances of CNR and RepCap in the composite score, we set a hyperparameter - setting it to 1 results in equal importance being given to both CNR and RepCap. Here we will use a value of 0.25; the optimal value to use for other devices will likely be different. In general, the higher the noise level, the higher the optimal CNR importance value.

Once we complete computing the composite scores, we can select the circuit with the highest composite score, and train it on the target dataset using a noiseless simulator. After training the circuit, we can evaluate it on the test portion of the target dataset before moving to running circuits on a real device. We do this to get a ballpark figure for the performance we can expect when we deploy the circuit on OQC Lucy, which is likely to be a little worse due to hardware noise.

In [53]:
circuits_save_dir = './candidate_circuits'
dataset = 'mnist_2'
encoding_type = 'angle'
num_data_reps = 1
device_name = 'ionq_harmony'
num_epochs = 200
batch_size = 256
num_qubits = 4
num_meas_qubits = 1
num_data_for_rep_cap = 32
num_params_for_rep_cap = 32
num_cdcs = 32
num_candidates_per_circ = 250
num_circs = 250
num_runs_per_circ = 5
noise_importance = 0.25
save_dir = './trained_circuit'
learning_rate = 0.01

In [54]:
train_elivagar_circuits(
    circuits_save_dir, dataset, encoding_type, 
    num_data_reps, device_name, num_epochs, batch_size,
    num_qubits, num_meas_qubits,
    num_data_for_rep_cap, num_params_for_rep_cap,
    num_cdcs, num_candidates_per_circ,
    num_circs, num_runs_per_circ, noise_importance,
    save_dir, learning_rate
)

0
[ 18 133  91 138 230 154 151 231  92  15 212 137  35  44  72 219 226   9
 238 152 143 221  20  56  99 239  97  75  48 166 134 163  53 234 184  55
  14 182 113  25   2 180  13  61 243 142 227 248 204 210  86 242 110 136
  49  85 170 188  63 124 103 222 235  68 195 111 244 121 208  67 114 216
 148  79 164  16 200  89  36  43  31 196   3  93 223 115  71  74 206 246
 186   0   7 118  82  38 131  95 211  17 193  32  23 168 236 129 140  47
 127 122 156  54 237 120 229  65  40 159  28 213 245  94  62  90 139 150
 187  88  78  59 185   6 207 172 132 162 165 169  96  42  34 167 109 178
 144 101  76 202 174   4 116 191 240  24   1 106 157 209 232 102  22 241
 194  12  29 135 130  37 171 107 146 125 145 228 225  52 233 203  83 173
  45 201 218 197 149  57 199  60  70 175 220  87 126 128  69 141  19 100
 181  33 183  81 249  21   8 205 176 215 123  73 117  26  39 105  50 247
 160  66   5 108 190 158 179  58 119  51  80 177  77  30  46 217  27 112
 198 192  64  41 224  84 104 153  10 161  11 214 

Epoch 147 | Step 1 | Loss: 0.22569 | Acc: 0.98438
Epoch 148 | Step 1 | Loss: 0.25521 | Acc: 0.97266
Epoch 149 | Step 1 | Loss: 0.21875 | Acc: 0.97656
Epoch 150 | Step 1 | Loss: 0.21007 | Acc: 0.97656
Epoch 151 | Step 1 | Loss: 0.25347 | Acc: 0.98047
Epoch 152 | Step 1 | Loss: 0.22049 | Acc: 0.98438
Epoch 153 | Step 1 | Loss: 0.25174 | Acc: 0.97266
Epoch 154 | Step 1 | Loss: 0.25521 | Acc: 0.97656
Epoch 155 | Step 1 | Loss: 0.24826 | Acc: 0.96875
Epoch 156 | Step 1 | Loss: 0.24306 | Acc: 0.97266
Epoch 157 | Step 1 | Loss: 0.20833 | Acc: 0.98047
Epoch 158 | Step 1 | Loss: 0.24306 | Acc: 0.97266
Epoch 159 | Step 1 | Loss: 0.22743 | Acc: 0.98438
Epoch 160 | Step 1 | Loss: 0.22049 | Acc: 0.97266
Epoch 161 | Step 1 | Loss: 0.23958 | Acc: 0.96875
Epoch 162 | Step 1 | Loss: 0.24132 | Acc: 0.97266
Epoch 163 | Step 1 | Loss: 0.21875 | Acc: 0.98438
Epoch 164 | Step 1 | Loss: 0.24826 | Acc: 0.97266
Epoch 165 | Step 1 | Loss: 0.22049 | Acc: 0.97266
Epoch 166 | Step 1 | Loss: 0.26389 | Acc: 0.96484


Epoch 113 | Step 1 | Loss: 0.23785 | Acc: 0.96875
Epoch 114 | Step 1 | Loss: 0.21181 | Acc: 0.98828
Epoch 115 | Step 1 | Loss: 0.21007 | Acc: 0.97266
Epoch 116 | Step 1 | Loss: 0.23438 | Acc: 0.98047
Epoch 117 | Step 1 | Loss: 0.24479 | Acc: 0.97656
Epoch 118 | Step 1 | Loss: 0.24653 | Acc: 0.96094
Epoch 119 | Step 1 | Loss: 0.22049 | Acc: 0.98828
Epoch 120 | Step 1 | Loss: 0.25174 | Acc: 0.97656
Epoch 121 | Step 1 | Loss: 0.26042 | Acc: 0.95703
Epoch 122 | Step 1 | Loss: 0.21875 | Acc: 0.99219
Epoch 123 | Step 1 | Loss: 0.23437 | Acc: 0.96875
Epoch 124 | Step 1 | Loss: 0.22222 | Acc: 0.98828
Epoch 125 | Step 1 | Loss: 0.21354 | Acc: 0.98047
Epoch 126 | Step 1 | Loss: 0.25174 | Acc: 0.96094
Epoch 127 | Step 1 | Loss: 0.23611 | Acc: 0.97656
Epoch 128 | Step 1 | Loss: 0.22569 | Acc: 0.98047
Epoch 129 | Step 1 | Loss: 0.22743 | Acc: 0.98047
Epoch 130 | Step 1 | Loss: 0.19097 | Acc: 0.98828
Epoch 131 | Step 1 | Loss: 0.24826 | Acc: 0.97266
Epoch 132 | Step 1 | Loss: 0.23264 | Acc: 0.98047


Epoch 78 | Step 1 | Loss: 0.25174 | Acc: 0.96484
Epoch 79 | Step 1 | Loss: 0.21181 | Acc: 0.98047
Epoch 80 | Step 1 | Loss: 0.22917 | Acc: 0.97266
Epoch 81 | Step 1 | Loss:    0.25 | Acc: 0.96875
Epoch 82 | Step 1 | Loss: 0.28299 | Acc: 0.98438
Epoch 83 | Step 1 | Loss: 0.22049 | Acc: 0.98438
Epoch 84 | Step 1 | Loss: 0.22569 | Acc: 0.98438
Epoch 85 | Step 1 | Loss: 0.26389 | Acc: 0.96484
Epoch 86 | Step 1 | Loss: 0.24479 | Acc: 0.97266
Epoch 87 | Step 1 | Loss: 0.25521 | Acc: 0.97656
Epoch 88 | Step 1 | Loss: 0.19965 | Acc: 0.99609
Epoch 89 | Step 1 | Loss:  0.2691 | Acc: 0.96484
Epoch 90 | Step 1 | Loss: 0.25521 | Acc: 0.96094
Epoch 91 | Step 1 | Loss: 0.21701 | Acc: 0.97266
Epoch 92 | Step 1 | Loss: 0.23264 | Acc: 0.98047
Epoch 93 | Step 1 | Loss: 0.26215 | Acc: 0.97266
Epoch 94 | Step 1 | Loss: 0.23438 | Acc: 0.97656
Epoch 95 | Step 1 | Loss: 0.35069 | Acc: 0.97656
Epoch 96 | Step 1 | Loss: 0.19965 | Acc: 0.98438
Epoch 97 | Step 1 | Loss: 0.24826 | Acc: 0.98438
Epoch 98 | Step 1 | 

Epoch 43 | Step 1 | Loss: 0.24479 | Acc: 0.96484
Epoch 44 | Step 1 | Loss: 0.21354 | Acc: 0.98047
Epoch 45 | Step 1 | Loss: 0.22917 | Acc: 0.98828
Epoch 46 | Step 1 | Loss: 0.24306 | Acc: 0.97656
Epoch 47 | Step 1 | Loss: 0.20833 | Acc: 0.98828
Epoch 48 | Step 1 | Loss: 0.21354 | Acc: 0.99219
Epoch 49 | Step 1 | Loss: 0.24653 | Acc: 0.97656
Epoch 50 | Step 1 | Loss: 0.21701 | Acc: 0.98438
Epoch 51 | Step 1 | Loss: 0.25694 | Acc: 0.95312
Epoch 52 | Step 1 | Loss: 0.24479 | Acc: 0.98828
Epoch 53 | Step 1 | Loss: 0.24479 | Acc: 0.95703
Epoch 54 | Step 1 | Loss: 0.22049 | Acc: 0.99219
Epoch 55 | Step 1 | Loss:  0.2309 | Acc: 0.97656
Epoch 56 | Step 1 | Loss: 0.26215 | Acc: 0.97266
Epoch 57 | Step 1 | Loss: 0.28993 | Acc: 0.95312
Epoch 58 | Step 1 | Loss: 0.23611 | Acc: 0.97656
Epoch 59 | Step 1 | Loss: 0.23785 | Acc: 0.97656
Epoch 60 | Step 1 | Loss: 0.20139 | Acc: 0.98438
Epoch 61 | Step 1 | Loss: 0.24826 | Acc: 0.96484
Epoch 62 | Step 1 | Loss: 0.34201 | Acc: 0.97656
Epoch 63 | Step 1 | 

Epoch 8 | Step 1 | Loss: 0.29861 | Acc: 0.95703
Epoch 9 | Step 1 | Loss:  0.2309 | Acc: 0.98438
Epoch 10 | Step 1 | Loss:    0.25 | Acc: 0.98047
Epoch 11 | Step 1 | Loss: 0.35417 | Acc: 0.97656
Epoch 12 | Step 1 | Loss: 0.25174 | Acc: 0.98047
Epoch 13 | Step 1 | Loss: 0.27083 | Acc: 0.96875
Epoch 14 | Step 1 | Loss: 0.27083 | Acc: 0.98828
Epoch 15 | Step 1 | Loss: 0.24826 | Acc: 0.98047
Epoch 16 | Step 1 | Loss: 0.35243 | Acc: 0.97266
Epoch 17 | Step 1 | Loss: 0.23438 | Acc: 0.97266
Epoch 18 | Step 1 | Loss: 0.22917 | Acc: 0.99219
Epoch 19 | Step 1 | Loss: 0.23958 | Acc: 0.96875
Epoch 20 | Step 1 | Loss: 0.27604 | Acc: 0.97656
Epoch 21 | Step 1 | Loss:    0.25 | Acc: 0.97266
Epoch 22 | Step 1 | Loss: 0.24132 | Acc: 0.96484
Epoch 23 | Step 1 | Loss: 0.27083 | Acc: 0.96484
Epoch 24 | Step 1 | Loss: 0.26563 | Acc: 0.96875
Epoch 25 | Step 1 | Loss: 0.23958 | Acc: 0.97656
Epoch 26 | Step 1 | Loss: 0.23611 | Acc: 0.97656
Epoch 27 | Step 1 | Loss:  0.2309 | Acc: 0.98438
Epoch 28 | Step 1 | Lo

Epoch 174 | Step 1 | Loss: 0.25174 | Acc: 0.96484
Epoch 175 | Step 1 | Loss: 0.34201 | Acc: 0.98438
Epoch 176 | Step 1 | Loss:    0.25 | Acc: 0.95703
Epoch 177 | Step 1 | Loss: 0.21354 | Acc: 0.98828
Epoch 178 | Step 1 | Loss: 0.25868 | Acc: 0.95703
Epoch 179 | Step 1 | Loss: 0.26563 | Acc: 0.96484
Epoch 180 | Step 1 | Loss: 0.19618 | Acc: 0.98047
Epoch 181 | Step 1 | Loss: 0.27431 | Acc: 0.96875
Epoch 182 | Step 1 | Loss:  0.1875 | Acc: 0.99219
Epoch 183 | Step 1 | Loss: 0.32813 | Acc: 0.98047
Epoch 184 | Step 1 | Loss: 0.22743 | Acc: 0.97656
Epoch 185 | Step 1 | Loss: 0.24826 | Acc: 0.97656
Epoch 186 | Step 1 | Loss: 0.22222 | Acc: 0.98047
Epoch 187 | Step 1 | Loss: 0.21528 | Acc: 0.98438
Epoch 188 | Step 1 | Loss: 0.37153 | Acc: 0.96875
Epoch 189 | Step 1 | Loss: 0.19618 | Acc: 0.98438
Epoch 190 | Step 1 | Loss: 0.26389 | Acc: 0.96484
Epoch 191 | Step 1 | Loss:  0.2309 | Acc: 0.97266
Epoch 192 | Step 1 | Loss: 0.26563 | Acc: 0.96484
Epoch 193 | Step 1 | Loss: 0.23438 | Acc: 0.97266


(0.2650270541141402, 0.9710000000000001)

## Evaluate circuit on target device

Finally, we can evaluate the circuit on our target hardware, the OQC Lucy device: